# Gradio Day!

Today we will build User Interfaces using the outrageously simple Gradio framework.

Prepare for joy!

Please note: your Gradio screens may appear in 'dark mode' or 'light mode' depending on your computer settings.

In [74]:
# imports

import os
import requests
from bs4 import BeautifulSoup
from typing import List
from dotenv import load_dotenv
from openai import OpenAI
import google.generativeai as genai
import anthropic
import time

In [75]:
import gradio as gr # oh yeah!

In [76]:
# Load environment variables in a file called .env
# Print the key prefixes to help with any debugging

load_dotenv(override=True)
openai_api_key = os.getenv('OPENAI_API_KEY')
anthropic_api_key = os.getenv('ANTHROPIC_API_KEY')
google_api_key = os.getenv('GOOGLE_API_KEY')
deepseek_api_key = os.getenv('DEEPSEEK_API_KEY')

if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")
    
if anthropic_api_key:
    print(f"Anthropic API Key exists and begins {anthropic_api_key[:7]}")
else:
    print("Anthropic API Key not set")

if google_api_key:
    print(f"Google API Key exists and begins {google_api_key[:8]}")
else:
    print("Google API Key not set")

if deepseek_api_key:
    print(f"Deepseek API Key exists and begins {deepseek_api_key[:8]}")
else:
    print("Google API Key not set")

OpenAI API Key exists and begins sk-proj-
Anthropic API Key exists and begins sk-ant-
Google API Key exists and begins AIzaSyBS
Deepseek API Key exists and begins sk-35138


In [77]:
# Connect to OpenAI, Anthropic and Google; comment out the Claude or Google lines if you're not using them

openai = OpenAI()

claude = anthropic.Anthropic()

google.generativeai.configure()

In [78]:
# A generic system message - no more snarky adversarial AIs!

system_message = "Tu eres un muy buen asistente"

In [79]:
# Let's wrap a call to GPT-4o-mini in a simple function

def message_gpt(prompt):
    messages = [
        {"role": "system", "content": system_message},
        {"role": "user", "content": prompt}
      ]
    completion = openai.chat.completions.create(
        model='gpt-4o-mini',
        messages=messages,
    )
    return completion.choices[0].message.content

In [80]:
# This can reveal the "training cut off", or the most recent date in the training data

message_gpt("Cual es la fecha de hoy?")

'Hoy es el 27 de octubre de 2023.'

## User Interface time!

In [81]:
# here's a simple function

def shout(text):
    print(f"Shout has been called with input {text}")
    return text.upper() #Devuelve el texto en mayuscula

In [82]:
shout("hello")

Shout has been called with input hello


'HELLO'

In [83]:
# The simplicty of gradio. This might appear in "light mode" - I'll show you how to make this in dark mode later.

gr.Interface(fn=shout, inputs="textbox", outputs="textbox").launch()

* Running on local URL:  http://127.0.0.1:7892

To create a public link, set `share=True` in `launch()`.


In [84]:
# Adding share=True means that it can be accessed publically
# A more permanent hosting is available using a platform called Spaces from HuggingFace, which we will touch on next week
# NOTE: Some Anti-virus software and Corporate Firewalls might not like you using share=True. If you're at work on on a work network, I suggest skip this test.

gr.Interface(fn=shout, inputs="textbox", outputs="textbox", flagging_mode="never").launch(share=True)

* Running on local URL:  http://127.0.0.1:7893

Could not create share link. Please check your internet connection or our status page: https://status.gradio.app.


In [85]:
# Adding inbrowser=True opens up a new browser window automatically

gr.Interface(fn=shout, inputs="textbox", outputs="textbox", flagging_mode="never").launch(inbrowser=True)

* Running on local URL:  http://127.0.0.1:7894

To create a public link, set `share=True` in `launch()`.


## Forcing dark mode

Gradio appears in light mode or dark mode depending on the settings of the browser and computer. There is a way to force gradio to appear in dark mode, but Gradio recommends against this as it should be a user preference (particularly for accessibility reasons). But if you wish to force dark mode for your screens, below is how to do it.

In [86]:
# Define this variable and then pass js=force_dark_mode when creating the Interface

force_dark_mode = """
function refresh() {
    const url = new URL(window.location);
    if (url.searchParams.get('__theme') !== 'dark') {
        url.searchParams.set('__theme', 'dark');
        window.location.href = url.href;
    }
}
"""
gr.Interface(fn=shout, inputs="textbox", outputs="textbox", flagging_mode="never", js=force_dark_mode).launch()

* Running on local URL:  http://127.0.0.1:7895

To create a public link, set `share=True` in `launch()`.


In [87]:
# Inputs and Outputs

view = gr.Interface(
    fn=shout,
    inputs=[gr.Textbox(label="Tu mensaje:", lines=6)],
    outputs=[gr.Textbox(label="Respuesta:", lines=8)],
    flagging_mode="never"
)
view.launch()

* Running on local URL:  http://127.0.0.1:7896

To create a public link, set `share=True` in `launch()`.


In [88]:
# And now - changing the function from "shout" to "message_gpt"

view = gr.Interface(
    fn=message_gpt,
    inputs=[gr.Textbox(label="Tu mensaje:", lines=6)],
    outputs=[gr.Textbox(label="Respuestae:", lines=8)],
    flagging_mode="never"
)
view.launch()

* Running on local URL:  http://127.0.0.1:7897

To create a public link, set `share=True` in `launch()`.


In [89]:
# Let's use Markdown
# Are you wondering why it makes any difference to set system_message when it's not referred to in the code below it?
# I'm taking advantage of system_message being a global variable, used back in the message_gpt function (go take a look)
# Not a great software engineering practice, but quite sommon during Jupyter Lab R&D!

system_message = "Eres un asistente muy útil que responde en Markdown"

view = gr.Interface(
    fn=message_gpt,
    inputs=[gr.Textbox(label="Tu mensaje:")],
    outputs=[gr.Markdown(label="Respuesta:")],
    flagging_mode="never"
)
view.launch()

* Running on local URL:  http://127.0.0.1:7898

To create a public link, set `share=True` in `launch()`.


In [90]:
# Let's create a call that streams back results
# If you'd like a refresher on Generators (the "yield" keyword),
# Please take a look at the Intermediate Python notebook in week1 folder.

def stream_gpt(prompt):
    messages = [
        {"role": "system", "content": system_message},
        {"role": "user", "content": prompt}
      ]
    stream = openai.chat.completions.create(
        model='gpt-4o-mini',
        messages=messages,
        stream=True
    )
    result = ""
    for chunk in stream:
        result += chunk.choices[0].delta.content or ""
        yield result

In [91]:
view = gr.Interface(
    fn=stream_gpt,
    inputs=[gr.Textbox(label="Tu mensaje:")],
    outputs=[gr.Markdown(label="Respuesta:")],
    flagging_mode="never"
)
view.launch()

* Running on local URL:  http://127.0.0.1:7899

To create a public link, set `share=True` in `launch()`.


In [92]:
def stream_claude(prompt):
    result = claude.messages.stream(
        model="claude-3-haiku-20240307",
        max_tokens=1000,
        temperature=0.7,
        system=system_message,
        messages=[
            {"role": "user", "content": prompt},
        ],
    )
    response = ""
    with result as stream:
        for text in stream.text_stream:
            response += text or ""
            yield response

In [93]:
view = gr.Interface(
    fn=stream_claude,
    inputs=[gr.Textbox(label="Tu mensaje:")],
    outputs=[gr.Markdown(label="Respuesta:")],
    flagging_mode="never"
)
view.launch()

* Running on local URL:  http://127.0.0.1:7900

To create a public link, set `share=True` in `launch()`.


## Minor improvement

I've made a small improvement to this code.

Previously, it had these lines:

```
for chunk in result:
  yield chunk
```

There's actually a more elegant way to achieve this (which Python people might call more 'Pythonic'):

`yield from result`

I cover this in more detail in the Intermediate Python notebook in the week1 folder - take a look if you'd like more.

In [94]:
def stream_gemini(prompt):
    """
    Genera una respuesta en streaming desde Gemini AI con una salida más fluida.
    """
    model = genai.GenerativeModel(model_name="gemini-1.5-flash")  # Cambia si usas otro modelo

    stream = model.generate_content(prompt, stream=True)  # Activa el stream

    result = ""
    for chunk in stream:
        if chunk.text:  # Verifica si hay contenido en el fragmento
            for char in chunk.text:  # Itera carácter por carácter
                result += char
                yield result  # Devuelve la respuesta progresivamente
                time.sleep(0.01)  # Pequeño retraso para hacer la animación más fluida

In [95]:
def stream_model(prompt, model):
    if model=="GPT":
        result = stream_gpt(prompt)
    elif model=="Claude":
        result = stream_claude(prompt)
    elif model=="Gemini":
        result = stream_gemini(prompt)
    else:
        raise ValueError("Unknown model")
    yield from result

In [96]:

view = gr.Interface(
    fn=stream_model,  # Función que maneja el modelo de streaming
    inputs=[
        gr.Textbox(label="Your message:"),  # Caja de texto para el usuario
        gr.Dropdown(  # Menú desplegable para seleccionar el modelo
            ["GPT", "Claude", "Gemini"],  # Opciones disponibles
            label="Select model",  # Etiqueta del dropdown
            value="GPT"  # Valor predeterminado
        )
    ],
    outputs=[gr.Markdown(label="Response:")],  # Respuesta en formato Markdown
    flagging_mode="never"  # Desactiva el modo de marcado de respuestas
)

view.launch()  # Inicia la interfaz web de Gradio

* Running on local URL:  http://127.0.0.1:7901

To create a public link, set `share=True` in `launch()`.


# Building a company brochure generator

Now you know how - it's simple!

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../important.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#900;">Before you read the next few cells</h2>
            <span style="color:#900;">
                Try to do this yourself - go back to the company brochure in week1, day5 and add a Gradio UI to the end. Then come and look at the solution.
            </span>
        </td>
    </tr>
</table>

In [98]:
# With massive thanks to Bill G. who noticed that a prior version of this had a bug! Now fixed.

system_message = "Eres un asistente que analiza el contenido de la página de inicio del sitio web de una empresa \
y crea un folleto breve sobre la compañía para posibles clientes, inversionistas y reclutadores. Responde en markdown."

In [149]:
def stream_gpt(user_message, tone):
    """
    Genera una respuesta en streaming desde GPT con tono personalizado.
    """
    system_message = generate_system_message(tone)  # Se genera el mensaje del sistema con el tono

    messages = [
        {"role": "system", "content": system_message},  # Incluir el mensaje del sistema con el tono
        {"role": "user", "content": user_message}
    ]

    stream = openai.chat.completions.create(
        model="gpt-4o",
        messages=messages,
        stream=True
    )

    result = ""
    for chunk in stream:
        if chunk.choices[0].delta.content:
            result += chunk.choices[0].delta.content
            yield result  # Devuelve el resultado en tiempo real


In [150]:
from IPython.display import Markdown, display, update_display

def stream_claude(user_message, tone):
    """
    Genera una respuesta en streaming desde Claude con tono personalizado y formato Markdown.
    """
    system_message = generate_system_message(tone)  # Generar el mensaje del sistema con el tono

    result = claude.messages.stream(
        model="claude-3-haiku-20240307",
        max_tokens=1000,
        temperature=0.7,
        system=system_message,
        messages=[{"role": "user", "content": user_message}],
    )

    response = ""
    disp = display(Markdown(""), display_id=True)  # Inicializa un display vacío en Markdown

    with result as stream:
        for text in stream.text_stream:
            response += text or ""  # Evita valores nulos
            clean_response = response.replace("```", "").replace("markdown", "")  # Limpia código de bloque
            update_display(Markdown(clean_response), display_id=disp.display_id)  # Actualiza en Markdown
            yield clean_response  # Devuelve la respuesta formateada en tiempo real


In [156]:
def stream_gemini(user_message, tone):
    """
    Genera una respuesta en streaming desde Gemini con tono personalizado.
    """
    system_message = generate_system_message(tone)  # Se genera el mensaje del sistema con el tono

    model = genai.GenerativeModel(model_name="gemini-1.5-flash")

    stream = model.generate_content(f"{system_message}\n\nUsuario: {user_message}", stream=True)

    result = ""
    for chunk in stream:
        if chunk.text:
            for char in chunk.text:  # Se muestra carácter por carácter para un efecto fluido
                result += char
                time.sleep(0.01)  # Pausa pequeña para un efecto más natural en la salida
                yield result
                


In [157]:
from urllib.parse import urlparse

class Website:
    """
    Representa una página web y extrae su contenido útil.
    """
    def __init__(self, url):
        # Asegurar que la URL tenga esquema ('http' o 'https')
        parsed_url = urlparse(url)
        if not parsed_url.scheme:
            url = "https://" + url  # Si no tiene esquema, agregar 'https://'

        self.url = url
        response = requests.get(url)  # Realiza la solicitud HTTP

        self.body = response.content
        soup = BeautifulSoup(self.body, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"

        # Eliminar elementos irrelevantes
        for irrelevant in soup.body(["script", "style", "img", "input"]):
            irrelevant.decompose()

        self.text = soup.body.get_text(separator="\n", strip=True)

    def get_contents(self):
        """
        Retorna el contenido relevante de la página.
        """
        return f"Webpage Title:\n{self.title}\nWebpage Contents:\n{self.text}\n\n"


In [158]:
def stream_brochure(nombre_empresa, url, modelo, tono):
    """
    Genera un brochure en streaming con el contenido de la landing page y el tono especificado.
    """
    # Generar el contenido de la página web
    contenido_pagina = Website(url).get_contents()

    # Crear el mensaje de sistema con el tono personalizado
    mensaje_sistema = f"Eres un asistente que analiza el contenido de la página de inicio de una empresa y genera un folleto breve en español para clientes, inversionistas y reclutadores. Responde en formato Markdown. Tu tono debe ser: {tono}."

    # Construir el prompt final
    prompt = f"{mensaje_sistema}\n\nPor favor, genera un folleto sobre {nombre_empresa}. Aquí está el contenido de su página web:\n\n{contenido_pagina}"

    # Seleccionar el modelo adecuado y hacer la llamada con streaming
    if modelo == "GPT":
        resultado = stream_gpt(prompt, tono)  # Se pasa el tono
    elif modelo == "Claude":
        resultado = stream_claude(prompt, tono)  # Se pasa el tono
    elif modelo == "Gemini":
        resultado = stream_gemini(prompt, tono)  # Se pasa el tono
    else:
        raise ValueError("Modelo desconocido. Debes seleccionar GPT, Claude o Gemini.")

    # Retornar la respuesta en streaming
    yield from resultado


In [ ]:
view = gr.Interface(
    fn=stream_brochure,  # Función que maneja el streaming del brochure
    inputs=[
        gr.Textbox(label="Nombre de la Compañía:"),  # Nombre de la empresa
        gr.Textbox(label="Página Web. URL"),  # URL de la página web
        gr.Dropdown(["GPT", "Claude", "Gemini"], label="Selecciona el modelo"),  # Selección del modelo
        gr.Textbox(label="Tono de respuesta:", placeholder="Ej. Amigable, Profesional, Sarcástico")  # Especificar tono
    ],
    outputs=[gr.Markdown(label="Brochure:")],  # Respuesta en formato Markdown
    flagging_mode="never"  # Desactiva el modo de marcado de respuestas
)

# 📌 Iniciar la aplicación de Gradio
view.launch()
